In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/armutdataset/armut_data.csv


In [2]:
pip install skimpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
  Attempting uninstall: polars
    Found existing installation: polars 1.5.0
    Uninstalling polars-1.5.0:
      Successfully uninstalled polars-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompa

In [3]:
from mlxtend.frequent_patterns import apriori,association_rules
from skimpy import skim

In [4]:
data=pd.read_csv("/kaggle/input/armutdataset/armut_data.csv")
pd.set_option("display.max_columns",None)

In [5]:
data.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
UserId,162523.0,13089.803862,7325.816060,0.0,6953.0,13139.0,19396.0,25744.0
ServiceId,162523.0,21.641140,13.774405,0.0,13.0,18.0,32.0,49.0
CategoryId,162523.0,4.325917,3.129292,0.0,1.0,4.0,6.0,11.0


In [7]:
num_col=[col for col in data.columns if data[col].dtype !="O"]
print("Numerik Sütun Sayısı:",len(num_col))
cat_col=[col for col in data.columns if data[col].dtype == "O"]
print("Categorik sÜTUN Sayısı:",len(cat_col))
print("Kaç kayıttan oluşuyor:",data.shape[0])
print("Kaç feature var?:",data.shape[1])
print("EKSİK DEĞER MEVCUT MU ?\n",data.isnull().sum())

Numerik Sütun Sayısı: 3
Categorik sÜTUN Sayısı: 1
Kaç kayıttan oluşuyor: 162523
Kaç feature var?: 4
EKSİK DEĞER MEVCUT MU ?
 UserId        0
ServiceId     0
CategoryId    0
CreateDate    0
dtype: int64


In [8]:
skim(data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 162523 │ │ int64       │ 3     │                                                          │
│ │ Number of columns │ 4      │ │ string      │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA   ┃ NA %   ┃ mean    ┃ sd      ┃ p0   ┃ p25    ┃ p50     ┃ p75     ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ UserId          │    0 │      0 │   13090 │    7326 │    0 │   6953 │   13140 │   19400 │  25740 │ ▇▇▇▇▇▇  │  │
│ │ ServiceId       │    0 │      0 │   21.64 │   13.77 │    0 │     13 │      18 │      32 │     49 │ ▅▅▇▂▃▃  │  │
│ │ CategoryId      │    0 │      0 │   4.326 │   3.129 │    0 │      1 │       4 │       6 │     11 │ ▅▁▇▃▁▂  │  │
│ └─────────────────┴──────┴────────┴─────────┴─────────┴──────┴────────┴─────────┴─────────┴────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ CreateDate                │       0 │          0 │                            2 │                   325046 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [10]:
# Adım 2: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir.
# ServiceID ve CategoryID'yi "_" ile birleştirerek hizmetleri temsil edecek yeni bir değişken oluşturunuz.

In [11]:
data["Hizmet"]=data["ServiceId"].astype(str)+"_"+data["CategoryId"].astype(str)

In [12]:
data.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [13]:

# Adım 3: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.
# Association Rule Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir.
# Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti;
# 2017’in 10.ayında aldığı  9_4, 38_4  hizmetleri başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir.
# Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_"
# ile birleştirirek ID adında yeni bir değişkene atayınız.

In [14]:
data["CreateDate"]=pd.to_datetime(data["CreateDate"])  # Tarih formatına döndürmek için to_datetime yapısını kullanırız.

In [15]:
# Sadece gün ve ay bilgilerini almak ve birleştirmek
data["New_Date"] = data["CreateDate"].dt.strftime('%Y-%m')  


# %Y --> 2017 şeklinde yazdırır.
# %y --> 17 şeklinde yazdırır.


In [16]:
data.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [17]:
data["SepetID"]=data["UserId"].astype(str)+"_"+data["New_Date"].astype(str)
# String'e ifadeye çevirerek işlem yaparız. - string bir ifadedir.


In [18]:
data.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [19]:

#########################
# GÖREV 2: Birliktelik Kuralları Üretiniz
#########################

# Adım 1: Aşağıdaki gibi sepet hizmet pivot table’i oluşturunuz.

In [20]:
pivot_table=data.groupby(["SepetID", "Hizmet"])["CategoryId"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
# Groupby ile her sepet için belirli hizmetin dağılımını görürüz.
# CategoryId sütunundaki öğelerin sayısını hesaplar.
# unstack ile hizmetleri sütun, sepetıdleri ise satır yaparız.
#Fillna ile eksik olan değerleri 0 ile doldururuz.
#applymap --> aply fonksiyonu satır ve sütun bilgisi vererek işlem yapar fakat applymap bütüm gözlemleri gezer. x 0'dan büyük ise 1 değil ise 0 yazar.


/tmp/ipykernel_18/1232916410.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table=data.groupby(["SepetID", "Hizmet"])["CategoryId"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)


In [21]:
pivot_table.head(10)

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
10000_2017-12,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10000_2018-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10001_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10001_2018-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# Adım 2: Birliktelik kurallarını oluşturunuz.

In [23]:
## Apriori Algoritması --> Belirli bir support etkisiyle ürün birlikteliğini ortaya koyar.
## Support --> Birlikte görülme olasılığını temsil eder.
## Confidence --> Biri alındığında diğerininde alınması olasılı, güven ilişkisini temsil eder.
## Lift --> Bir öğenin diğerine olan etkisini , yükselişi temsil eder. Öğeler arasındaki ilişkini gücüde diyebiliriz.


frequent_itemsets=apriori(pivot_table,min_support=0.01,use_colnames=True)


/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [24]:
## Association Rules --> Veri içindeki örüntüleri bulmamızı sağlayan kural tabanlı bir yaklaşımdır. Belirli bir öğenin diğer 
# öğelerle birlikte ne kadar sık alındığını gösterir.
rules= association_rules(frequent_itemsets,
                         metric="support",
                         min_threshold=0.01)

# metric--> kuralı oluştururken support değerinin dikkate alınacağını belirtir.
# min_threshold --> Belirlediğimiz oran sıklığında veya daha fazla sıklıkta görülen kurallar dikkate alınır.


In [25]:
rules.sort_values(by="support", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
12,(22_0),(2_0),0.047515,0.130286,0.016568,0.348700,2.676409,0.010378,1.335350,0.657611
13,(2_0),(22_0),0.130286,0.047515,0.016568,0.127169,2.676409,0.010378,1.091260,0.720197
14,(25_0),(2_0),0.042895,0.130286,0.013437,0.313257,2.404371,0.007849,1.266432,0.610268
15,(2_0),(25_0),0.130286,0.042895,0.013437,0.103136,2.404371,0.007849,1.067168,0.671590
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047
5,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728


In [26]:


#Adım 3: arl_recommender fonksiyonunu kullanarak
#en son 2_0 hizmetini alan bir kullanıcıya
#hizmet önerisinde bulununuz.

In [27]:
def arl_recommender(rules, Hizmet, rec_count=1):
    sorted_rules = rules.sort_values("lift", ascending=False) #3 Lift'e göre sıralama yaparız.
    recommendation_list = [] # Boş bir liste oluşturuyoruz. 
    for i, service in enumerate(sorted_rules["antecedents"]): # i indeksi temsil eder, service ise her bir antecedents(hizmet) değerini temsil eder.
        for j in list(service):
            if j == Hizmet:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0]) ## sorted_rules'ten i indeksini seç ve consequents sütununa erişiriz.İlk değeri alırız.
                # Consequents bize antecedents için beklenen sonucu gösterir.
                # Cons. ve antecedent gibi değerler set,series gibi yapılarda olur.Listeye dönüştürererk daha koaly erişim sağlayabiliriz.

    return recommendation_list[0:rec_count]

In [28]:
arl_recommender(rules, '2_0', rec_count=5)


['22_0', '25_0', '15_1', '13_11', '38_4']

# 